In [1]:
#! /usr/bin/env python
#coding=utf-8
import pycurl
import StringIO
from lxml import etree
import os
import re
import json
import time
import random
from urllib import quote

import requests

In [3]:
def get_html(url):
    #url = 'http://blog.csdn.net/li6727975/article/details/46126079'
    #url = 'https://www.upwork.com/o/profiles/browse/?page=%s' % str(page)
    useragent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:43.0) Gecko/20100101 Firefox/43.0"
    s = StringIO.StringIO()
    crl = pycurl.Curl()
    #crl.setopt(pycurl.PROXY, '60.210.104.254:808')
    #crl.setopt(pycurl.PROXY, 'http://127.0.0.1:1080')
    crl.setopt(pycurl.URL, url)
    crl.setopt(pycurl.REFERER, url)
    crl.setopt(pycurl.FOLLOWLOCATION, True)
    crl.setopt(pycurl.TIMEOUT, 60)
    crl.setopt(pycurl.ENCODING, 'gzip')
    crl.setopt(pycurl.USERAGENT, useragent)
    crl.setopt(pycurl.NOSIGNAL, True)
    crl.setopt(pycurl.WRITEFUNCTION, s.write)
    crl.perform()
    html = s.getvalue() 
    #tree = etree.HTML(html)
    return html

In [4]:
def get_data(html_text):
    
    tree_text = etree.HTML(html_text)
    #parse data
    name = tree_text.xpath('//*[@id="optimizely-header-container-default"]/div[1]/div[1]/div/div[2]/h2/span')[0].text
    city = tree_text.xpath('//*[@id="optimizely-header-container-default"]/div[1]/div[1]/div/div[2]/div/strong')[0].text
    country = tree_text.xpath('//*[@id="optimizely-header-container-default"]/div[1]/div[1]/div/div[2]/div/strong')[1].text
    occupation = tree_text.xpath('//*[@id="optimizely-header-container-default"]/div[2]/h3/span')[0].text
    summary = tree_text.xpath('//*[@id="optimizely-header-container-default"]/div[2]/div/p')[0].text.strip()
    
    rate_hour = re.findall('"hourlyRate".*?amount":(.*?)}',html_text)[0]
    skills = re.findall('"br_data.search_term":"(.*?)"',html_text)[0].split('|')
    link = re.findall('href="(.*?)"\S',html_text)[0]
    
    one ={}
    one['name']=name
    one['link']=link
    one['location']=city+','+country
    one['occupation']=occupation
    one['summary'] = summary
    one['rate_hour']=rate_hour
    one['skills']=skills
    return one

In [ ]:
from bs4 import BeautifulSoup
import re

In [ ]:
soup = BeautifulSoup(html_text)
print(soup.prettify())

In [ ]:
url = 'https://www.upwork.com/o/companies/~01900fbf5e8dc8a019/'

In [5]:
url = 'https://www.upwork.com/o/profiles/users/_~01434d6e0b74340d34/'

In [4]:
url = 'https://www.upwork.com/o/profiles/users/_~01a5f38c9d2059d8fc/'

In [6]:
res = get_html_requests(url)

In [7]:
res.content

'<html><head><script src="https://www.google.com/recaptcha/api.js"></script>                    <link href="https://assets.static-upwork.com/components/1.8.4/air.global.1.8.4.min.css" rel="stylesheet">\n                    <link href="https://assets.static-upwork.com/components/1.8.4/air.components.1.8.4.min.css" rel="stylesheet">\n                    <link href="https://assets.static-upwork.com/fonts/1.10.0/fonts.gotham.1.10.0.css" rel="stylesheet">\n                    <link href="https://assets.static-upwork.com/fonts/1.10.0/fonts.air-icons.1.10.0.css" rel="stylesheet">\n                    <script src="https://assets.static-upwork.com/components/1.8.4/core.1.8.4.min.js"></script>\n                    <script src="https://assets.static-upwork.com/components/1.8.4/angular-select.1.8.4.min.js"></script>\n                    <script src="https://assets.static-upwork.com/components/1.8.4/files.1.8.4.min.js"></script>\n                    <script>\n                        //async font lo

In [8]:
data = get_data(res.content)

IndexError: list index out of range

In [5]:
html_text = get_html(url)
data = get_data(html_text)

In [6]:
for i in data:
    print i+' : '+ str(data[i])

name : Karlene C.
skills : ['general-office-skills', 'microsoft-excel', 'accounting', 'computer-skills', 'financial-reporting']
summary : Providing accounting and bookkeeping service to small businesses. Treating you with courtesy, integrity, protect your privacy. Guaranteed an efficient, honest accounting service that achieves remarkable results for you. My 33 years of experience and notable expertise ensure that your bookkeeping is up to date and accurate. And most of all, I want you to be able to say; "I love my accountant."
rate_hour : 17
link : https://www.upwork.com/o/profiles/users/_~01a5f38c9d2059d8fc/
location : yuma,United States
occupation : KC's Bookkeeping & Tax Service


In [20]:
# not always 10, moving company links
def get_link_list(page,keyword):
    #have to quote keyword
    url = 'https://www.upwork.com/o/profiles/browse/?page=%s&q=%s' % (str(page),quote(keyword))
    urls = []
    html_text = get_html(url)
    tree_text = etree.HTML(html_text)
    for one in tree_text.xpath('//section[@data-qa]'):
        res = one.xpath('./div/article/div[2]/h4/a')[0]
        root = 'https://www.upwork.com'
        link = root+res.xpath('@href')[0]
        # get rid of 'companies'
        if "profiles" in link:
            urls.append(link)
    return urls

In [23]:
#getting keywords
keywords=['machine learning','java']

In [ ]:
for keyword in keywords:
    for p in range(1, 501):
        urls = get_link_list(p,keyword)
        for url in urls:
            html_text = get_html(url)
            data = get_data(html_text)
            # saving data

In [24]:
# record what have done. 